### Test N2 from alchemy

##### Calculate path H$_2$ $\rightarrow$ N$_2$, since then $\epsilon_{\text{h}}$ always well defined

$\Delta E$ (H$_2$ $\rightarrow$ N$_2$) = -110.39

$\Delta E_{\rm{ref}}$ = -107.71

$\Delta E (\rm{H}_2 \rightarrow \rm{N}_2)- \Delta E_{\rm{ref}}$ = -2.69

deviation from reference value somewhat smaller than for vac $\rightarrow$ N$_2$ (-3.14 Ha), but could also be because the path is shorter

##### What is the error for integration between adjacent elements?
around 400mHa for everyting except for intervals involving He2
largest error for intervals involving He2 
sudden change in e_homo from He2 to Li2 probably not main problem because for H2$\rightarrow$He2 large error as well although change in e_homo very smooth

e_homo is not the main issue

#### difference between two different alchemical energies
Error drastically reduced (-0.5 Ha instead of -3.14 Ha)

look at error per step (vac H2 the largest because no canceling of wrong dissociation behaviour?)

#### SI
integration by trapezoidal rule

$\Delta E$ (H$_2$ $\rightarrow$ N$_2$) = -110.56

$\Delta E (\rm{H}_2 \rightarrow \rm{N}_2)- \Delta E_{\rm{ref}}$ = -2.85

In [1]:
import numpy as np
import scipy as sc
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib
from pyscf.data import nist
import qml
from matplotlib import pyplot as plt
from IPython.display import display, Markdown


angstrom = 1 / 0.52917721067

In [2]:
def generate_mol(el, d, basis, total_elec, num_elec):
    mol = pyscf.gto.Mole()
    mol.atom = f'{el} 0 0 0; {el} 0 0 {d}'
    mol.basis = basis
    mol.charge = total_elec - num_elec
    mol.verbose = 0
    mol.build()
    return(mol)

def pyscf_APDFT(mol, total_elec):
    method = "HF"
    if method not in ["CCSD", "HF"]:
        raise NotImplementedError("Method %s not supported." % method)

    deltaZ = [-mol.charge/2, -mol.charge/2]
    includeonly = np.arange(len(mol.atom_coords()))

    def add_qmmm(calc, mol, deltaZ):
        mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]/ angstrom, deltaZ)

        def energy_nuc(self):
            q = mol.atom_charges().astype(np.float)
            q[includeonly] += deltaZ
            return mol.energy_nuc(q)

        mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

        return mf

    if method == "HF":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        dm1_ao = calc.make_rdm1()
        total_energy = calc.e_tot
    if method == "CCSD":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        mycc = pyscf.cc.CCSD(calc).run()
        dm1 = mycc.make_rdm1()
        dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
        total_energy = mycc.e_tot

    # GRIDLESS, as things should be ############################
    # Total energy of SCF run

#     print("TOTAL_ENERGY", total_energy)

    # Electronic EPN from electron density
    alchemical_potentials = []
    for site in includeonly:
        mol.set_rinv_orig_(mol.atom_coords()[site])
#         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
        alchemical_potentials.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())

    # energy of HOMO and LUMO
    nocc = int((total_elec - mol.charge)/2)
    e_homo = calc.mo_energy[nocc-1]
    e_lumo = calc.mo_energy[nocc]

    return(alchemical_potentials, e_homo, e_lumo, calc.energy_nuc())

def sample_path(num_elecs, element, distance, basis, total_nelec):
    alchemical_potentials = []
    e_homos = []
    e_lumos = []
    e_nucs = []

    for num_elec in num_elecs:
        mol = generate_mol(element, distance, basis, total_nelec, num_elec)
        alchpot, ehomo, elumo, enuc = pyscf_APDFT(mol, total_nelec)
        alchemical_potentials.append(alchpot)
        e_homos.append(ehomo)
        e_lumos.append(elumo)
        e_nucs.append(enuc)
    alchemical_potentials = -1*np.array(alchemical_potentials)
    e_homos = np.array(e_homos)
    e_lumos = np.array(e_lumos)
    e_nucs = np.array(e_nucs)
    return(alchemical_potentials, e_homos, e_lumos, e_nucs)

def get_energy(lam_vals, alchemical_potentials, nuc_rep, e_homos, dZ, dN, int_rule = 'simp'):
    if int_rule == 'simp':
        mu_I = sc.integrate.simps(alchemical_potentials, lam_vals)
    elif int_rule == 'trapz':
        mu_I = np.trapz(alchemical_potentials, lam_vals)
    e_homo = np.trapz(e_homos, lam_vals)
    
    return(2*mu_I * dZ + e_homo*dN + nuc_rep)

def get_lam_vals(num_elecs):
    return(1/(num_elecs[-1]-num_elecs[0])*num_elecs - num_elecs[0]/(num_elecs[-1]-num_elecs[0]))


### Referenz calculations:  H$_2$, N$_2$

In [3]:
def ref_hf(el, d, basis = 'def2-qzvp'):
    mol = pyscf.gto.Mole()
    mol.atom = f'{el} 0 0 0; {el} 0 0 {d}'
    mol.basis = basis
    mol.build()
    m = pyscf.scf.RHF(mol)
    m.kernel()
    e_ref_qzvp = m.e_tot
    return(e_ref_qzvp)

def get_ref_dict(d, basis = 'def2-qzvp'):
    eref_dict = {'vac':0, 'H2':0, 'He2':0, 'Li2':0, 'Be2':0, 'B2':0, 'C2':0, 'N2':0}

    for k in eref_dict.keys():
        if k != 'vac':
            eref_dict[k] = ref_hf(k.strip('2'), d, basis = basis)
    return(eref_dict)

In [ ]:
eref_d1p5 = get_ref_dict(1.5)

In [ ]:
eref_d10 = get_ref_dict(10.0)

In [4]:
eref_H2 = ref_hf('H', 1.5)
eref_N2 = ref_hf('N', 1.5)
eref_H2_d10 = ref_hf('H', 10.0)
eref_N2_d10 = ref_hf('N', 10.0)

converged SCF energy = -1.00484447618229
converged SCF energy = -108.711062429174
converged SCF energy = -0.74100524112954
converged SCF energy = -107.782256769781


#### H$_2$ $\rightarrow$ N$_2$

In [ ]:
element = 'N'
distance = 1.5
basis = 'def2-qzvp'
total_nelec = 14
num_elecs = np.array([2, 4, 6, 8, 10, 12, 14])
lam_vals = 1/(num_elecs[-1]-num_elecs[0])*num_elecs - num_elecs[0]/(num_elecs[-1]-num_elecs[0])
alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path(num_elecs, element, distance, basis, total_nelec)
e_H2N_simp = get_energy(lam_vals, alchemical_potentials[:,0], e_nucs[-1], e_homos, 6, 12)
e_H2N_trapz = get_energy(lam_vals, alchemical_potentials[:,0], e_nucs[-1], e_homos, 6, 12, 'trapz')

In [ ]:
e_homos = np.concatenate((np.zeros(1), e_homos))

In [ ]:

fig, ax = plt.subplots(1,1)

ax.plot(np.arange(0, 16, 2), e_homos, '-o')

In [ ]:
print('integration by trapezoidal rule')
display(Markdown(r'$\Delta E$ (H$_2$ $\rightarrow$ N$_2$) = ' + str(e_H2N_trapz)))
display(Markdown(r'$\Delta E_{\rm{ref}}$ = ' + str(eref_N2 - eref_H2)))
display(Markdown(r'$\Delta E (\rm{H}_2 \rightarrow \rm{N}_2)- \Delta E_{\rm{ref}}$ = ' + str(e_H2N_trapz-(eref_N2 - eref_H2))))

print('integration by simpsons rule')
display(Markdown(r'$\Delta E$ (H$_2$ $\rightarrow$ N$_2$) = ' + str(e_H2N_simp)))
display(Markdown(r'$\Delta E_{\rm{ref}}$ = ' + str(eref_N2 - eref_H2)))
display(Markdown(r'$\Delta E (\rm{H}_2 \rightarrow \rm{N}_2)- \Delta E_{\rm{ref}}$ = ' + str(e_H2N_simp-(eref_N2 - eref_H2))))

#### Pairs of adjacent elements

In [ ]:
element = 'N'
distance = 1.5
basis = 'def2-qzvp'
total_nelec = 14
num_elecs = np.array([0,2])
pairs = {'vactoH2':[], 'H2toHe2':[], 'He2toLi2':[], 'Li2toBe2':[], 'Be2toB2':[], 'B2toC2':[], 'C2toN2':[]}
alchs = []
ehs = []
for k in pairs.keys():
    if k == 'vactoH2':
        alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path([num_elecs[-1]], element, distance, basis, total_nelec)
        alchemical_potentials = np.concatenate((np.array([[0.0, 0.0]]), alchemical_potentials))
        e_homos = np.concatenate((np.array([0]), e_homos))
        e_lumos = np.concatenate((np.array([0]), e_lumos))
        e_nucs = np.concatenate((np.array([0.0]), e_nucs))     
    else:
        alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path(num_elecs, element, distance, basis, total_nelec)
    alchs.append(alchemical_potentials)
    ehs.append(e_homos)
    dN = num_elecs[-1] - num_elecs[0]
    dZ = dN/2
    lam_vals = get_lam_vals(num_elecs)
    e_X2toY2_simp = get_energy(lam_vals, alchemical_potentials[:,0], e_nucs[-1], e_homos, dZ, dN) - e_nucs[0]
    pairs[k].append(e_X2toY2_simp)
    num_elecs = num_elecs + 2 #raise to get correct number for next pair of diatomics

In [ ]:
# add reference delta
for k in pairs.keys():
    diatomics = k.split('to')
    pairs[k].append(eref_d1p5[diatomics[1]]-eref_d1p5[diatomics[0]])
    pairs[k].append(pairs[k][0]-pairs[k][1])


In [ ]:
sum_ = 0
for k in pairs.keys():
    sum_ += pairs[k][0]
sum_

In [ ]:
plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 16
fig, ax = plt.subplots(1,1)
x = np.arange(1, 15, 2)
dE = []
for k in pairs.keys():
    dE.append(pairs[k][2])
ax.plot(x, dE, '-o')
ax.set_xticks(x)
xstring = [r'vac$\rightarrow$H2', r'H2$\rightarrow$He2', r'He2$\rightarrow$Li2', r'Li2$\rightarrow$Be2', r'Be2$\rightarrow$B2', r'B2$\rightarrow$C2', r'C2$\rightarrow$N2']
ax.set_xticklabels(xstring, fontsize=16)
ax.set_ylabel(r'Error $\Delta E$ (Ha)')
# ax.plot(np.arange(0, 16, 2), e_homos, '-o')

largest error for intervals involving He2 
sudden change in e_homo from He2 to Li2 probably not main problem because for H2$\rightarrow$He2 large error as well although change in e_homo very smooth

# $\Delta$ H$_2$ $\rightarrow$ N$_2$, d = 1.5 and d = 5.0

In [ ]:
e_H2N_dist = {1.5:0, 10.0:0}
for d in e_H2N_dist.keys():
    element = 'N'
    distance = d
    basis = 'def2-qzvp'
    total_nelec = 14
    num_elecs = np.array([2, 4, 6, 8, 10, 12, 14])
    lam_vals = 1/(num_elecs[-1]-num_elecs[0])*num_elecs - num_elecs[0]/(num_elecs[-1]-num_elecs[0])
    alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path(num_elecs, element, distance, basis, total_nelec)
    e_H2N_simp = get_energy(lam_vals, alchemical_potentials[:,0], e_nucs[-1], e_homos, 6, 12)
    e_H2N_dist[d] = e_H2N_simp

In [ ]:
d_e_H2N = e_H2N_dist[1.5] - e_H2N_dist[10.0]
d_eref_H2N = (eref_N2 - eref_H2) - (eref_N2_d10 - eref_H2_d10)

print('integration by simpsons rule')
display(Markdown(r'$\Delta \Delta E$ (H$_2$ $\rightarrow$ N$_2$) = ' + str(d_e_H2N)))
display(Markdown(r'$\Delta \Delta E_{\rm{ref}}$ = ' + str(d_eref_H2N)))
display(Markdown(r'$\Delta \Delta E (\rm{H}_2 \rightarrow \rm{N}_2)- \Delta \Delta E_{\rm{ref}}$ = ' + str(d_e_H2N-d_eref_H2N)))

In [ ]:
e_H2N_dist

#### vac $\rightarrow$ N$_2$, d = 1.5 and d = 5.0

10 ang unstable

In [40]:
e_vac2N_dist = {1.5:0, 5.0:0}
for d in e_vac2N_dist.keys():
    element = 'N'
    distance = d
    basis = 'def2-qzvp'
    total_nelec = 14
    num_elecs = np.array([2, 4, 6, 8, 10, 12, 14])
    alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path(num_elecs, element, distance, basis, total_nelec)
    
    lam_vals = 1/num_elecs[-1]*num_elecs
    lam_vals = np.concatenate((np.array([0.0]), lam_vals))
    alchemical_potentials = np.concatenate((np.array([[0.0, 0.0]]), alchemical_potentials))
    e_homos = np.concatenate((np.array([0.0]), e_homos))
    e_lumos = np.concatenate((np.array([0.0]), e_lumos))
    
    e_vac2N_simp = get_energy(lam_vals, alchemical_potentials[:,0], e_nucs[-1], e_homos, 7, 14)
    e_vac2N_dist[d] = e_vac2N_simp

In [41]:
d_e_vac2N = e_vac2N_dist[1.5] - e_vac2N_dist[5.0]
d_eref_N = eref_N2 - eref_N2_d10

print('integration by simpsons rule')
display(Markdown(r'$\Delta \Delta E$ (vac $\rightarrow$ N$_2$) = ' + str(d_e_vac2N)))
display(Markdown(r'$\Delta \Delta E_{\rm{ref}}$ = ' + str(d_eref_N)))
display(Markdown(r'$\Delta \Delta E (vac \rightarrow \rm{N}_2)- \Delta \Delta E_{\rm{ref}}$ = ' + str(d_e_vac2N-d_eref_N)))

integration by simpsons rule


$\Delta \Delta E$ (vac $\rightarrow$ N$_2$) = -1.375093810204163

$\Delta \Delta E_{\rm{ref}}$ = -0.9288056593931486

$\Delta \Delta E (vac \rightarrow \rm{N}_2)- \Delta \Delta E_{\rm{ref}}$ = -0.44628815081101436

In [39]:
e_vac2N_dist

{1.5: -111.8519489910883, 5.0: -110.47685512228774}

#### difference for adjacent pairs

In [ ]:
element = 'N'
distance = 1.5
basis = 'def2-qzvp'
total_nelec = 14
num_elecs = np.array([0,2])
pairs = {'vactoH2':[], 'H2toHe2':[], 'He2toLi2':[], 'Li2toBe2':[], 'Be2toB2':[], 'B2toC2':[], 'C2toN2':[]}
alchs = []
ehs = []
for k in pairs.keys():
    if k == 'vactoH2':
        alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path([num_elecs[-1]], element, distance, basis, total_nelec)
        alchemical_potentials = np.concatenate((np.array([[0.0, 0.0]]), alchemical_potentials))
        e_homos = np.concatenate((np.array([0]), e_homos))
        e_lumos = np.concatenate((np.array([0]), e_lumos))
        e_nucs = np.concatenate((np.array([0.0]), e_nucs))     
    else:
        alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path(num_elecs, element, distance, basis, total_nelec)
    alchs.append(alchemical_potentials)
    ehs.append(e_homos)
    dN = num_elecs[-1] - num_elecs[0]
    dZ = dN/2
    lam_vals = get_lam_vals(num_elecs)
    e_X2toY2_simp = get_energy(lam_vals, alchemical_potentials[:,0], e_nucs[-1], e_homos, dZ, dN) - e_nucs[0]
    pairs[k].append(e_X2toY2_simp)
    num_elecs = num_elecs + 2 #raise to get correct number for next pair of diatomics

In [ ]:
element = 'N'
distance = 10.0
basis = 'def2-qzvp'
total_nelec = 14
num_elecs = np.array([0,2])
pairs10 = {'vactoH2':[], 'H2toHe2':[], 'He2toLi2':[], 'Li2toBe2':[], 'Be2toB2':[], 'B2toC2':[], 'C2toN2':[]}
alchs = []
ehs = []
for k in pairs10.keys():
    if k == 'vactoH2':
        alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path([num_elecs[-1]], element, distance, basis, total_nelec)
        alchemical_potentials = np.concatenate((np.array([[0.0, 0.0]]), alchemical_potentials))
        e_homos = np.concatenate((np.array([0]), e_homos))
        e_lumos = np.concatenate((np.array([0]), e_lumos))
        e_nucs = np.concatenate((np.array([0.0]), e_nucs))     
    else:
        alchemical_potentials, e_homos, e_lumos, e_nucs = sample_path(num_elecs, element, distance, basis, total_nelec)
    alchs.append(alchemical_potentials)
    ehs.append(e_homos)
    dN = num_elecs[-1] - num_elecs[0]
    dZ = dN/2
    lam_vals = get_lam_vals(num_elecs)
    e_X2toY2_simp = get_energy(lam_vals, alchemical_potentials[:,0], e_nucs[-1], e_homos, dZ, dN) - e_nucs[0]
    pairs10[k].append(e_X2toY2_simp)
    num_elecs = num_elecs + 2 #raise to get correct number for next pair of diatomics

In [ ]:
dE_vactoN2_dict = dict.fromkeys(pairs.keys())
for k in pairs.keys():
    dE_vactoN2_dict[k] = []
    dE_vactoN2_dict[k].append(pairs[k][0] - pairs10[k][0])

In [ ]:
# add reference delta
for k in dE_vactoN2_dict.keys():
    diatomics = k.split('to')
    dE_ref1p5 = eref_d1p5[diatomics[1]]-eref_d1p5[diatomics[0]]
    dE_ref10 = eref_d10[diatomics[1]]-eref_d10[diatomics[0]]
    
    dE_vactoN2_dict[k].append(dE_ref1p5-dE_ref10)
    
    dE_vactoN2_dict[k].append(dE_vactoN2_dict[k][0] - dE_vactoN2_dict[k][1])

In [ ]:
dE_vactoN2_dict